# Intro to two-dimensional plotting with Makie

This tutorial introduces two-dimensional static and animated visualization
with GLMakie.jl, using data from a freely-decaying barotropic turbulence simulation.

We'll

1. Create a simple figure with one axis
2. Create a figure with layout
3. Update a plot live while a simulation runs
4. Record an animation while a simulation runs
5. Use `Slider` to explore data in a static plot in post
6. Use `Observable` to animate data in post with a fancy `colorrange`

If you're using a notebook or you forgot to write `julia --project`,
these lines will help...

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `~/Projects/ClimaWorkshops/intro-makie`
┌ Warning: The active manifest file is an older format with no julia version entry. Dependencies may have been resolved with a different julia version.
└ @ nothing /Users/gregorywagner/Projects/ClimaWorkshops/intro-makie/Manifest.toml:0


Now we import all the packages we're planning to use: Makie with the OpenGL backend,
Oceananigans, the function `mean`, and `Printf` for pretty printing.

In [2]:
using GLMakie
using Oceananigans
using Oceananigans.Simulations: reset!
using Statistics: mean
using Printf

# The setup: freely-decaying barotropic turbulence on the beta plane

All of the following examples use the setup below, which simulates
barotropic turbulence on the beta-plane in a meridionally-bounded domain.

In [3]:
# Set up a simulation with random velocity initial conditions
grid = RectilinearGrid(size=(128, 128), extent=(2π, 2π), halo=(3, 3), topology=(Periodic, Bounded, Flat))
model = NonhydrostaticModel(; grid, advection=WENO5(), coriolis=BetaPlane(f₀=1, β=20))
ϵ(x, y, z) = 2rand() - 1
set!(model, u=ϵ, v=ϵ)
simulation = Simulation(model, Δt=0.02, stop_iteration=2000)

┌ Warning: defaulting to uniform WENO scheme with Float64 precision, use WENO5(grid = grid) if this was not intended
└ @ Oceananigans.Advection /Users/gregorywagner/.julia/packages/Oceananigans/9HQkq/src/Advection/weno_fifth_order.jl:154


Simulation of NonhydrostaticModel{CPU, RectilinearGrid}(time = 0 seconds, iteration = 0)
├── Next time step: 20 ms
├── Elapsed wall time: 0 seconds
├── Stop time: Inf years
├── Stop iteration : 2000.0
├── Wall time limit: Inf
├── Callbacks: OrderedDict with 4 entries:
│   ├── stop_time_exceeded => Callback of stop_time_exceeded on IterationInterval(1)
│   ├── stop_iteration_exceeded => Callback of stop_iteration_exceeded on IterationInterval(1)
│   ├── wall_time_limit_exceeded => Callback of wall_time_limit_exceeded on IterationInterval(1)
│   └── nan_checker => Callback of NaNChecker for u on IterationInterval(100)
├── Output writers: OrderedDict with no entries
└── Diagnostics: OrderedDict with no entries

## Diagnostics: vorticity, speed, mean momentum, and enstrophy

We build some diagnostics so we have data to plot:

In [4]:
# Vorticity
u, v, w = model.velocities
ζ = compute!(Field(∂x(v) - ∂y(u)))
xζ, yζ, zζ = nodes(ζ)

# Speed
s_op = @at (Center, Center, Center) sqrt(u^2 + v^2)
s = compute!(Field(s_op))
xs, ys, zs = nodes(s)

# Zonal-averaged enstrophy and momentum
Z = Field(Average(ζ^2, dims=1))
U = Field(Average(u, dims=1))
compute!(Z)
compute!(U)

1×128×1 Field{Nothing, Center, Center} reduced over dims = (1,) on RectilinearGrid on CPU
├── data: OffsetArrays.OffsetArray{Float64, 3, Array{Float64, 3}}, size: (1, 128, 1)
├── grid: 128×128×1 RectilinearGrid{Float64, Periodic, Bounded, Flat} on CPU with 3×3×0 halo
├── operand: mean! over dims (1,) of 128×128×1 Field{Face, Center, Center} on RectilinearGrid on CPU
└── status: time=0.0

Note: you'll find the function `interior` used extensively in the code below.
This function returns a `view` into the data underlying a `Field`.
For example,

In [5]:
typeof(interior(ζ, :, :, 1))

SubArray{Float64, 2, Array{Float64, 3}, Tuple{UnitRange{Int64}, UnitRange{Int64}, Int64}, false}

returns a two-dimensional (x, y) view into the vorticity.

# Demo

## Create a simple figure with one axis

In [ ]:
fig = Figure(resolution=(800, 600))
ax = Axis(fig[1, 1], xlabel="x", ylabel="y", title="Vorticity", aspect=1)
hm = contourf!(ax, xζ, yζ, interior(ζ, :, :, 1), levels=5)
save("barotropic_turbulence_vorticity.png", fig)
display(fig)

## Create a figure with layout

In [ ]:
fig = Figure(resolution=(1200, 1200))

ax_ζ = Axis(fig[1, 1], xlabel="x", ylabel="y", title="Vorticity", aspect=1)
ax_s = Axis(fig[2, 1], xlabel="x", ylabel="y", title="Speed", aspect=1)
ax_Z = Axis(fig[1, 2], xlabel="Zonally-averaged enstrophy", ylabel="y")
ax_U = Axis(fig[2, 2], xlabel="Zonally-averaged zontal momentum", ylabel="y")

lbl = Label(fig[0, :], "Barotropic turbulence at t = 0")

cr_ζ = heatmap!(ax_ζ, xζ, yζ, interior(ζ, :, :, 1), colormap=:redblue)
cr_s = heatmap!(ax_s, xs, ys, interior(s, :, :, 1))
ln_Z = lines!(ax_Z, interior(Z, 1, :, 1), yζ)
ln_U = lines!(ax_U, interior(U, 1, :, 1), ys)

xlims!(ax_U, -0.1, 0.1)
xlims!(ax_Z, 0.0, 10.0)

save("barotropic_turbulence_vorticity_speed.png", fig)
display(fig)

## Update a plot live while a simulation runs

In [ ]:
function update!(sim)
    lbl.text[] = @sprintf("Barotropic turbulence at t = %.2f", time(sim))
    cr_ζ.input_args[3][] = interior(compute!(ζ), :, :, 1)
    cr_s.input_args[3][] = interior(compute!(s), :, :, 1)
    compute!(Z); compute!(U)
    ln_Z.input_args[1][] = interior(Z, 1, :, 1)
    ln_U.input_args[1][] = interior(U, 1, :, 1)
    return nothing
end

simulation.callbacks[:plot] = Callback(update!, IterationInterval(10))
run!(simulation)

## Record an animation

In [ ]:
pop!(simulation.callbacks, :plot)
reset!(simulation) # back to time=0, iteration=0
set!(model, u=ϵ, v=ϵ)

record(fig, "barotropic_turbulence_online.mp4", 1:100, framerate=24) do frame
    [time_step!(simulation) for i = 1:10]
    update!(simulation)
end

## Use a slider to explore data

In [ ]:
reset!(simulation) # back to time=0, iteration=0
set!(model, u=ϵ, v=ϵ)
simulation.stop_iteration = 1000
simulation.output_writers[:fields] = JLD2OutputWriter(model, (; ζ, s, Z, U),
                                                      schedule = IterationInterval(10),
                                                      prefix = "barotropic_turbulence",
                                                      force = true)

run!(simulation)

filename = "barotropic_turbulence.jld2"
ζ_ts = FieldTimeSeries(filename, "ζ")
s_ts = FieldTimeSeries(filename, "s")
Z_ts = FieldTimeSeries(filename, "Z")
U_ts = FieldTimeSeries(filename, "U")

Nt = length(ζ_ts.times)

fig = Figure(resolution=(1200, 1200))

ax_ζ = Axis(fig[1, 1], xlabel="x", ylabel="y", aspect=1)
ax_s = Axis(fig[2, 1], xlabel="x", ylabel="y", aspect=1)
ax_Z = Axis(fig[1, 2], xlabel="Zonally-averaged enstrophy", ylabel="y")
ax_U = Axis(fig[2, 2], xlabel="Zonally-averaged zontal momentum", ylabel="y")

slider = Slider(fig[3, :], range=1:Nt, startvalue=1)
n = slider.value
#n = Observable(1) # This works too if we don't need a slider

ζn = @lift interior(ζ_ts[$n], :, :, 1)
sn = @lift interior(s_ts[$n], :, :, 1)
Zn = @lift interior(Z_ts[$n], 1, :, 1)
Un = @lift interior(U_ts[$n], 1, :, 1)

smax = maximum(abs, interior(s_ts))

Dynamic colorange

In [ ]:
Navg = 30
ζlims = @lift begin
    if $n > Nt - Navg
        ζmax = maximum(abs, ζ_ts[$n])
        ζlim = ζmax / 2
    else
        ζmax = mean(maximum(abs, ζ_ts[nn]) for nn in $n:$n+Navg-1)
        ζlim = ζmax / 2
    end
    (-ζlim, ζlim)
end

hm_ζ = heatmap!(ax_ζ, xζ, yζ, ζn, colormap=:redblue, colorrange=ζlims)
hm_s = heatmap!(ax_s, xs, ys, sn, colorrange=(0, smax/2))

lines!(ax_Z, Zn, yζ)
lines!(ax_U, Un, ys)

xlims!(ax_U, -0.15, 0.15)

# TODO: make this pretty
# Colorbar(fig[2, 0], hm_s, label="Speed", flipaxis=false)
# Colorbar(fig[1, 0], hm_ζ, label="Vorticity", flipaxis=false)

title = @lift "Barotropic turbulence at t = " * string(ζ_ts.times[$n])
lbl = Label(fig[0, :], title)

display(fig)

## Create an animation in post-process

In [ ]:
record(fig, "barotropic_turbulence_offline.mp4", 1:100, framerate=24) do nn
    n[] = nn
    Zmax = maximum(Z_ts[nn])
    xlims!(ax_Z, -Zmax/10, 2Zmax)
end

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*